# Scraping Steam reviews for God Of War

https://steamcommunity.com/app/1593500/reviews/?browsefilter=toprated&snr=1_5_100010_&p=1

In [11]:
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import numpy as np

In [2]:
url = r"https://steamcommunity.com/app/1593500/reviews/?browsefilter=toprated&snr=1_5_100010_&p=1"

r = requests.get(url)
r.status_code

200

In [3]:
soup = bs(r.content, "html.parser")
review_divs = soup.findAll("div", class_="apphub_UserReviewCardContent") # reviews are on this div class

In [4]:
reviews = []
for div in review_divs:
    review_text = div.get_text().strip() # get div text and strip leading and trailing whitespace
    review_text = review_text.replace("\t", "") # strip remaining tabs from middle of text
    review_text = review_text.replace("\r", "") # strip carriage returns
    review_text = "\n".join(filter(None, review_text.split("\n"))) # strip line swaps while leaving one for further processing
    reviews.append(review_text)

In [6]:
reviews[0].split("\n") # reviewing the output

['1,839 people found this review helpful75 people found this review funny64',
 'Recommended',
 '46.3 hrs on record',
 'Posted: 1 November, 2022',
 "“Don't Be Sorry, Be Better.”"]

In [19]:
# creating an empty DF for storing data
df = pd.DataFrame({"Found helpful": pd.Series(dtype="int"),
                  "Recommend": pd.Series(dtype="str"),
                  "Hours on record": pd.Series(dtype="float"),
                  "Date posted": pd.Series(dtype="str"),
                  "Review text": pd.Series(dtype="str")}
                  )
# next we need to parse the information in reviews and fill the df

In [140]:
# creating regex patterns to parse out the data

pattern_helpfull = r"\d+(?:,\d{3})*" # this will result in a list of numbers and we only want the "found helpful" number so we need to slice the list with [0]
pattern_hours = r"([\d.]+)" # also a list but with only one value, so slicing with [0] also

# we wont need a regex pattern for the date posted string since it starts with "Posted: " so we can use string.split("Posted: ")[1]



In [154]:
# Testing regex
test = reviews[0].split("\n")[0]
test_hours = reviews[0].split("\n")[2]

result_test = re.findall(pattern_helpfull, test)[0].replace(",", "") # also replacing the dot
result_test_h = re.findall(pattern_hours, test_hours)[0]

print(test)
print(result_test)
print(test_hours)
print(result_test_h)

1,839 people found this review helpful75 people found this review funny64
1839
46.3 hrs on record
46.3


In [ ]:
#TODO loop that assigns the data to variables and puts appends them to the created df